In [1]:
import os, glob, obspy, sys, datetime
print('Current working directory is %s' % os.getcwd())
HOME = os.getenv('HOME')
sys.path.append(os.path.join(HOME, 'src','kitchensinkGT','LIB'))
from libseisGT import read_raspshakenet, write_stream2sds
stationXmlFile = 'rs.xml'
mseedfile = 'rs.mseed'
centerLon = -80.649
centerLat = 28.573
searchRadiusDeg = 0.2
launchtime=obspy.UTCDateTime('2022-09-05T02:09:40.000000Z')
inv, st = read_raspshakenet(launchtime-60, launchtime+120, mseedfile, stationXmlFile, \
                centerLat, centerLon, searchRadiusDeg)

print(inv)
if not st:
    st = obspy.read(mseedfile)
#write_stream2sds(st, SDS_TOP)

from libseisGT import create_trace_inventory, merge_inventories
ondate = obspy.UTCDateTime(2016,1,1)
my_inv1 = create_trace_inventory(st[0], netname='RShake', sitename='Roberts Building', net_ondate=ondate, \
                           sta_ondate=ondate, lat=centerLat, lon=centerLon, elev=2.0, depth=0.0, stationXml='rs2.xml')
print(my_inv1)
#st[0].id = 'AM.BCHH.00.HHZ'
my_inv2 = create_trace_inventory(st[1], netname='RShake', sitename='Roberts Building', net_ondate=ondate, \
                           sta_ondate=ondate, lat=centerLat, lon=centerLon, elev=2.0, depth=0.0, stationXml='rs3.xml')
print(my_inv2)

st2=st.copy()
st2[0].id = 'AM.BCHH.00.HHZ'
my_inv3 = create_trace_inventory(st2[0], netname='RShake', sitename='Beach House', net_ondate=ondate, \
                           sta_ondate=ondate+86400, lat=centerLat+0.05, lon=centerLon-0.03, elev=1.0, depth=1.0, stationXml='rs4.xml')
print(my_inv3)

st2[1].id = 'FL.DVEL.00.HHN'
my_inv4 = create_trace_inventory(st2[1], netname='Florida', sitename='Devils Elbow', net_ondate=ondate-86400, \
                           sta_ondate=ondate-86400, lat=centerLat+0.06, lon=centerLon-0.01, elev=5.0, depth=1.0, stationXml='rs5.xml')
print(my_inv4)

Current working directory is /Users/thompsong/src/GitHub/kitchensinkGT/PROJECTS/ROCKETSEIS/artemis
Trying to load inventory from 2022/09/05 02:08 to 2022/09/05 02:08
inventory saved to rs.xml
Inventory created at 2022-12-08T16:48:39.204106Z
	Sending institution: SeisComP (RaspberryShake)
	Contains:
		Networks (1):
			AM
		Stations (1):
			AM.R1E5E (Raspberry Shake Citizen Science Station)
		Channels (3):
			AM.R1E5E.00.EHZ, AM.R1E5E.00.EHN, AM.R1E5E.00.EHE
Writing inventory to rs2.xml
Inventory created at 2022-12-08T16:48:39.431610Z
	Created by: ObsPy 1.3.0
		    https://www.obspy.org
	Sending institution: Glenn Thompson
	Contains:
		Networks (1):
			AM
		Stations (1):
			AM.R1E5E (Roberts Building)
		Channels (1):
			AM.R1E5E.00.EHE
Writing inventory to rs3.xml
Inventory created at 2022-12-08T16:48:41.776609Z
	Created by: ObsPy 1.3.0
		    https://www.obspy.org
	Sending institution: Glenn Thompson
	Contains:
		Networks (1):
			AM
		Stations (1):
			AM.R1E5E (Roberts Building)
		Channe

In [2]:
my_inv = my_inv1.copy()
merge_inventories(my_inv, my_inv2)
merge_inventories(my_inv, my_inv3)
merge_inventories(my_inv, my_inv4)
print(my_inv)
my_inv.write('my_inv.xml', format="stationxml", validate=True) 
my_inv.write('my_inv.css', format="CSS")
netcodes = [this_net.code for this_net in my_inv.networks]
for net in ['AM', 'FL']:
    if net in [this_net.code for this_net in my_inv.networks]: #netcodes:
        print(net, netcodes.index(net))
    else:
        print(net, -1)

Merging station
Adding new channel EHN
Adding new station BCHH
Inventory created at 2022-12-08T16:48:39.431610Z
	Created by: ObsPy 1.3.0
		    https://www.obspy.org
	Sending institution: Glenn Thompson
	Contains:
		Networks (2):
			AM, FL
		Stations (3):
			AM.BCHH (Beach House)
			AM.R1E5E (Roberts Building)
			FL.DVEL (Devils Elbow)
		Channels (4):
			AM.BCHH.00.HHZ, AM.R1E5E.00.EHN, AM.R1E5E.00.EHE, FL.DVEL.00.HHN
AM 0
FL 1


In [3]:
their_inv = my_inv1.copy()
their_inv.extend(my_inv2)
their_inv.extend(my_inv3)
their_inv.extend(my_inv4)
print(their_inv)

Inventory created at 2022-12-08T16:49:05.835825Z
	Created by: ObsPy 1.3.0
		    https://www.obspy.org
	Sending institution: Glenn Thompson
	Contains:
		Networks (4):
			AM (3x), FL
		Stations (4):
			AM.BCHH (Beach House)
			AM.R1E5E (Roberts Building) (2x)
			FL.DVEL (Devils Elbow)
		Channels (4):
			AM.BCHH.00.HHZ, AM.R1E5E.00.EHN, AM.R1E5E.00.EHE, FL.DVEL.00.HHN


In [ ]:
print(inv.get_contents())

In [124]:
import pickle
from obspy.clients.nrl import NRL
nrl = NRL()

def write_response(fname, response):
    fptr = open(fname, 'wb')
    pickle.dump(response, fptr)
    fptr.close()

def read_response(fname):
    fptr = open(fname, 'rb')
    response = pickle.load(fptr)
    fptr.close()
    return response

def L22():
    # as for PASSCAL experiment at KSC - need to check values though
    response = nrl.get_response( \
        sensor_keys=['Sercel/Mark Products','L-22D','2200 Ohms','10854 Ohms'], \
        datalogger_keys=['REF TEK','RT 130 & 130-SMA','32','200'])
    write_response('L22.pkl', response)
    return response

def RS1D_50Hz():
    response = nrl.get_response( \
        sensor_keys=['Raspberry Shake (formerly OSOP)', 'Raspberry Shake 1D Racotech'], \
        datalogger_keys=['Raspberry Shake (formerly OSOP)', 'Raspberry Shake 1D', '50'])
    write_response('RS1D_50Hz.pkl', response)
    return response

def Trillium(samplerate=100):
    response = nrl.get_response( \
        sensor_keys=['Nanometrics','Trillium Compact 120 (Vault, Posthole, OBS)','754 V/m/s'], \
        datalogger_keys=['Nanometrics','Centaur','40 Vpp (1)','Off','Linear phase',str(samplerate)] )
    write_response('Trillum_%dHz.pkl' % samplerate, response)
    return response

def Chaparral(p2p=40, samplerate=100):
    if p2p==40:
        p2pstr = '40 Vpp (1)'
    elif p2p==1:
        p2pstr = '1 Vpp (40)'
    VperPa=0.4
    response = nrl.get_response( 
        sensor_keys=['Chaparral Physics','25','Low: 0.4 V/Pa'], \
        datalogger_keys=['Nanometrics','Centaur',p2pstr,'Off','Linear phase',str(samplerate)] )
    # workaround bug that stage 1 not applied for infrasound sensors
    response.instrument_sensitivity.value *= VperPa
    write_response('Chaparral_%dV_%dHz.pkl' % (p2p, samplerate), response)
    return response

def infraBSU(p2p=40, inches=0.5, samplerate=100):
    if p2p==40:
        p2pstr = '40 Vpp (1)'
    elif p2p==1:
        p2pstr = '1 Vpp (40)'
    if inches==0.5:
        inches_str = '46 microV/Pa / 0.5 inches'
        VperPa = 46e-6
    elif inches==1:
        inches_str = '23 microV/Pa / 1 inches'
        VperPa = 23e-6
    elif inches==5:
        # shouldn't this be 4.6 microV/Pa?
        inches_str = '9.1 microV/Pa / 5 inches' 
        VperPa = 4.6e-6
    response = nrl.get_response( 
        sensor_keys=['Johnson Infrasound','infraBSU (version 1)','0.048 Hz / 62.5 microns (Blue)', \
                                              inches_str], \
        datalogger_keys=['Nanometrics','Centaur',p2pstr,'Off','Linear phase',str(samplerate)] )
    # workaround bug that stage 1 not applied for infrasound sensors
    response.instrument_sensitivity.value *= VperPa
    write_response('infraBSU_%.1f_%dV_%dHz.pkl' % (inches, p2p, samplerate), response)
    return response

print(RS1D_50Hz())
print(Trillium())
print(Trillium(samplerate=250))
print(infraBSU())
print(infraBSU(p2p=1, samplerate=250))
print(Chaparral())
print(Chaparral(p2p=1, samplerate=250))

Channel Response
	From M/S (Velocity in Meters per Second) to COUNTS (Digital Counts)
	Overall Sensitivity: 4.69129e+08 defined at 5.000 Hz
	4 stages:
		Stage 1: PolesZerosResponseStage from M/S to V, gain: 1
		Stage 2: ResponseStage from V to V, gain: 1
		Stage 3: CoefficientsTypeResponseStage from V to COUNTS, gain: 4.69087e+08
		Stage 4: CoefficientsTypeResponseStage from COUNTS to COUNTS, gain: 1
Channel Response
	From M/S (Velocity in Meters per Second) to COUNTS (Digital Counts)
	Overall Sensitivity: 3.0172e+08 defined at 1.000 Hz
	6 stages:
		Stage 1: PolesZerosResponseStage from M/S to V, gain: 754.3
		Stage 2: PolesZerosResponseStage from V to V, gain: 1
		Stage 3: CoefficientsTypeResponseStage from V to COUNTS, gain: 400000
		Stage 4: CoefficientsTypeResponseStage from COUNTS to COUNTS, gain: 1
		Stage 5: CoefficientsTypeResponseStage from COUNTS to COUNTS, gain: 1
		Stage 6: CoefficientsTypeResponseStage from COUNTS to COUNTS, gain: 1
Channel Response
	From M/S (Velocity in 

In [125]:
import os, glob
response_files = glob.glob('*.pkl')
for response_file in response_files:
    response = read_response(response_file)
    print(response_file)
    print(response)
    print()

Trillum_250Hz.pkl
Channel Response
	From M/S (Velocity in Meters per Second) to COUNTS (Digital Counts)
	Overall Sensitivity: 3.0172e+08 defined at 1.000 Hz
	6 stages:
		Stage 1: PolesZerosResponseStage from M/S to V, gain: 754.3
		Stage 2: PolesZerosResponseStage from V to V, gain: 1
		Stage 3: CoefficientsTypeResponseStage from V to COUNTS, gain: 400000
		Stage 4: CoefficientsTypeResponseStage from COUNTS to COUNTS, gain: 1
		Stage 5: CoefficientsTypeResponseStage from COUNTS to COUNTS, gain: 1
		Stage 6: CoefficientsTypeResponseStage from COUNTS to COUNTS, gain: 1

RS1D_50Hz.pkl
Channel Response
	From M/S (Velocity in Meters per Second) to COUNTS (Digital Counts)
	Overall Sensitivity: 4.69129e+08 defined at 5.000 Hz
	4 stages:
		Stage 1: PolesZerosResponseStage from M/S to V, gain: 1
		Stage 2: ResponseStage from V to V, gain: 1
		Stage 3: CoefficientsTypeResponseStage from V to COUNTS, gain: 4.69087e+08
		Stage 4: CoefficientsTypeResponseStage from COUNTS to COUNTS, gain: 1

infraB

In [ ]:
# For Montserrat stations - try to figure out appropriate settings
from obspy.clients.nrl import NRL
nrl = NRL()

print('Most short period sensors were Mark L4. No idea how to represent analog telemetry of McVCO though')
print(nrl.sensors['Sercel/Mark Products']['L-4C'])
print()

print('Most broadband sensors were Guralp CMG-40T. Not sure if 30s - 100Hz or 30s - 50 Hz. Probably former.')
print(nrl.sensors['Guralp']['CMG-40T']['30s - 50 Hz'])
print()

print('Broadband dataloggers were Earth Data, but model unknown')
print(nrl.dataloggers['Earth Data'])
print()

print('Locko had Guralp CMG-3T at Leeds, which are lower noise. Should work for Stromboli, New Zealand, and Bali data')
print(nrl.sensors['Guralp']['CMG-3T'])
print()

print('MULTIMO Project possibly had this digitizer CMG-DM24 and used with a CMG-3T plus barometers')
print(nrl.dataloggers['Guralp']['CMG-DM24'])
print()

print('Combined Guralp CMG-40T with CMG-DM24 = CMG-6TD?')
print(nrl.dataloggers['Guralp']['CMG-6TD'])